In [1]:
!pip install -U tensorflow==2.10 
!pip install whisperx
!pip install googletrans==4.0.0-rc1

ERROR: Could not find a version that satisfies the requirement tensorflow==2.10 (from versions: 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==2.10


In [2]:
!pip install protobuf==3.19.*
!export PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

zsh:1: no matches found: protobuf==3.19.*


In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
import random
import cv2
from pathlib import Path
import uuid
import shutil
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import whisperx
from datetime import timedelta
import os
import time

/opt/anaconda3/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [5]:
import requests
import torch
from PIL import Image
from transformers import GPT2TokenizerFast, ViTImageProcessor, pipeline
from tqdm import tqdm
from tensorflow import keras
# set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [6]:
# функция, определяющая, является ли строка URL-адресом или нет
def is_url(string):
    try:
        result = parse.urlparse(string)
        return all([result.scheme, result.netloc, result.path])
    except:
        return False
    
# фукнция загрузки изображения
def load_image(image_path):
    if is_url(image_path):
        return Image.open(requests.get(image_path, stream=True).raw)
    elif os.path.exists(image_path):
        return Image.open(image_path)
        
# функция инференса
def get_caption(model, image_processor, tokenizer, image_path):
    image = load_image(image_path)
    # предобработка
    img = image_processor(image, return_tensors="pt").to(device)
    # генерируем описание
    output = model.generate(**img)
    # декодим вывод
    caption = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
    return caption

In [7]:
import torch
import whisperx

device = "cuda" if torch.cuda.is_available() else "cpu"

compute_type = "int8"
whisper_model = whisperx.load_model("large-v2", device=device, compute_type=compute_type)

def transcribe_video(input_video):
    batch_size = 32

    audio = whisperx.load_audio(input_video)
    result = whisper_model.transcribe(audio, batch_size=batch_size, language="ru")

    model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
    result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

    segments = result['segments']
    texts = []
    for seg in segments:
        texts.append(seg['text'])
    return ' '.join(texts)



Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.2.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.3.0. Bad things might happen unless you revert torch to 1.x.


In [8]:
from googletrans import Translator

translator = Translator()

def translate_frames_caption(source_texts):
    translated_texts = []
    for res in source_texts:
        text = translator.translate(res, src='en', dest='ru')
        translated_texts.append(text.text)
    return translated_texts

In [9]:
# функция, определяющая, является ли строка URL-адресом или нет
def is_url(string):
    try:
        result = parse.urlparse(string)
        return all([result.scheme, result.netloc, result.path])
    except:
        return False


# фукнция загрузки изображения
def load_image(image_path):
    if is_url(image_path):
        return Image.open(requests.get(image_path, stream=True).raw)
    elif os.path.exists(image_path):
        return Image.open(image_path)


In [10]:
# Параметр, отвечающий за то, куда будем складывать файлы
BASE_PATH = '../'
# Параметр, отвечающий за то, сколько фреймов резать за секунду
SAVING_FRAMES_PER_SECOND = .5


def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    """Функция, которая возвращает список длительностей, в которые следует сохранять кадры."""
    s = []
    # получаем продолжительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # используйте np.arange () для выполнения шагов с плавающей запятой
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s


def create_temp_directory_with_frames(video_file):
    uid = uuid.uuid1()
    dirname_str = BASE_PATH + str(uid) + '-opencv'
    dirname = Path(dirname_str)
    # создаем папку по названию видео файла
    if not os.path.isdir(dirname_str):
        dirname.mkdir(exist_ok=True)
    # создаем папку по названию видео файла
    # читать видео файл
    capture = cv2.VideoCapture(video_file)
    # получить FPS видео
    fps = capture.get(cv2.CAP_PROP_FPS)
    # если SAVING_FRAMES_PER_SECOND выше видео FPS, то установите его на FPS (как максимум)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей для сохранения
    saving_frames_durations = get_saving_frames_durations(capture, saving_frames_per_second)
    # запускаем цикл
    count = 0
    frame_insert_count = 1
    while True:
        is_read, frame = capture.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем продолжительность, разделив количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую раннюю продолжительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры длительности сохранены
            break
        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна длительности кадра,
            # затем сохраняем фрейм
            # frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            # only_file_name = filename.split("/")[-1]
            uid_frame = uuid.uuid1()
            frame_name = f"frame_{uid_frame}.jpg"
            frame_insert_count += 1

            cv2.imwrite(os.path.join(dirname_str, frame_name), frame)
            # удалить точку продолжительности из списка, так как эта точка длительности уже сохранена
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить количество кадров
        count += 1
    return dirname_str


In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# модель кодировщика, которая обрабатывает изображение и возвращает его фичи

# encoder_model = "WinKawaks/vit-small-patch16-224"
# encoder_model = "google/vit-base-patch16-224"
# encoder_model = "google/vit-base-patch16-224-in21k"
encoder_model = "microsoft/swin-base-patch4-window7-224-in22k"

# модель декодера, которая обрабатывает фичи с изображения и генерирует текст подписи

# decoder_model = "bert-base-uncased"
# decoder_model = "prajjwal1/bert-tiny"
decoder_model = "gpt2"

# Инициализируем токенайзер
# tokenizer = AutoTokenizer.from_pretrained(decoder_model)
tokenizer = GPT2TokenizerFast.from_pretrained(decoder_model)
# tokenizer = BertTokenizerFast.from_pretrained(decoder_model)
# Загружаем обработчик изображений
image_processor = ViTImageProcessor.from_pretrained(encoder_model)

# Используя pipeline api
image_captioner = pipeline("image-to-text", model="Abdou/vit-swin-base-224-gpt2-image-captioning")
image_captioner.model = image_captioner.model.to(device)


# функция инференса
def get_caption_by_image(model, image_processor, tokenizer, image_path):
    image = load_image(image_path)
    # предобработка
    img = image_processor(image, return_tensors="pt").to(device)
    # генерируем описание
    output = model.generate(**img)
    # декодим вывод
    return tokenizer.batch_decode(output, skip_special_tokens=True)[0]


# функция для получения краткого описания
def get_short_description(string):
    i = 255
    if len(string) > 255:
        while i > 0:
            if len(string) > 255:
                if string[i] not in [' ', '.', ',']:
                    i -= 1
                else:
                    break
    return string[:i]


def get_video_caption(video_file):
    directory_name = create_temp_directory_with_frames(video_file)
    # объявим массив, в который будем складывать результаты предсказаний по фреймам
    full_english_descriptions = []
    for dirname, _, filenames in os.walk(directory_name):
        for filename in filenames:
            full_name = os.path.join(dirname, filename)
            file_english_description = get_caption_by_image(image_captioner.model, image_processor, tokenizer, full_name)
            full_english_descriptions.append(file_english_description)
    # удаляем temp директорию
    shutil.rmtree(directory_name)

    # избавляемся от явных дублей
    full_english_descriptions = list(set(full_english_descriptions))

    full_russian_descriptions = translate_frames_caption(full_english_descriptions)
    descriptions_length = len(full_russian_descriptions) - 1

    random_frame_number = random.randint(0, descriptions_length)

    random_russian_description = full_russian_descriptions[random_frame_number]
    random_english_description = full_english_descriptions[random_frame_number]

    full_description_en = ' '.join(full_english_descriptions)
    full_description_ru = ' '.join(full_russian_descriptions)

    return {
        'description_ru': full_description_ru,
        'short_description_ru': random_russian_description,
        'description_en': full_description_en,
        'short_description_en': random_english_description
    }


In [12]:
invalided_links = {
    'link': [],
    'row_number': []
}


links = []
descriptions_ru = []
short_descriptions_ru = []
descriptions_en = []
short_descriptions_en = []
tags = []
texts = []

def enrich(row):
    start_time = time.time()
    link = row['link']
    result_caption = get_video_caption(link)
    text = transcribe_video(link)
    result_caption['text'] = text
    print("--- Вот за столько секунд обработали %s  запись ---" % (time.time() - start_time))
    return result_caption


def try_to_enrich(row, index):
    link = row['link']
    try:
        result = enrich(row)
        links.append(link)
        tags.append(row['description'])
        descriptions_ru.append(result['description_ru'])
        short_descriptions_ru.append(result['short_description_ru'])
        descriptions_en.append(result['description_en'])
        short_descriptions_en.append(result['short_description_en'])
        texts.append(result['text'])
    except Exception as inst:
        print('УПАЛИИИ')
        print(type(inst))
        print(inst.args)
        print(inst)
        invalided_links['link'].append(link)
        invalided_links['row_number'].append(index)

In [13]:
DATASET_PATH = './2024_400k.csv'
dataset = pd.read_csv(DATASET_PATH).copy()

batch = 10
for i in range(0, 50, batch):
    j = 0
    while j < batch:
        current_number = i + j
        row = dataset.iloc[current_number]
        try_to_enrich(row, current_number)
        print(current_number)
        j += 1

Some weights of the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jonatasgrosman/wav2vec2-large-xlsr-53-russian and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
Y

--- Вот за столько секунд обработали 17.595946073532104  запись ---
0


KeyboardInterrupt: 

In [ ]:
len(texts)

In [ ]:
pd.DataFrame(data={
        'link': links,
        'tags': tags,
        'description_ru': descriptions_ru,
        'short_description_ru': short_descriptions_ru,
        'description_en': descriptions_en,
        'short_description_en': short_descriptions_en,
        'text': texts
}).head()

In [ ]:
DATASET_TEMP_PATH = '/kaggle/working/test.csv'
DATASET_FAIL_PATH = '/kaggle/working/fail.csv'
pd.DataFrame(data={
        'link': links,
        'tags': tags,
        'description_ru': descriptions_ru,
        'short_description_ru': short_descriptions_ru,
        'description_en': descriptions_en,
        'short_description_en': short_descriptions_en,
        'text': texts
}).to_csv(DATASET_TEMP_PATH, index=False)

pd.DataFrame(data={
    'link': invalided_links['link'],
    'row_number': invalided_links['row_number'],
}).to_csv(DATASET_FAIL_PATH, index=False)